In [11]:
from langchain_community.document_loaders import JSONLoader

loader = JSONLoader(
    file_path="../data/oms-central/oms-central-processed.json",
    jq_schema='.',
    text_content=False
)

docs = loader.load()
docs[0]

Document(metadata={'source': '/Users/hanqihua/Desktop/learning-assistant/data/oms-central/oms-central-processed.json', 'seq_num': 1}, page_content='[{\'name\': \'Special Topics: Applied Natural Language Processing\', \'description\': \'The primary objective of this course is to introduce you to broad classes of techniques and tools for analyzing text data using Natural Language Processing (NLP) algorithms and techniques. It emphasizes on how to apply pre-processing, processing, and post processing NLP techniques to analyze and develop NLP models.\', \'syllabus\': {\'url\': \'https://mahdi-roozbahani.github.io/ANLP-Summer23/\'}, \'isFoundational\': False, \'slug\': \'special-topics-applied-natural-language-processing\', \'codes\': [\'CSE-8803-OAN\'], \'reviewCount\': 7, \'rating\': 3.857142857142857, \'difficulty\': 2.857142857142857, \'workload\': 12.142857142857142}, {\'name\': \'Special Topics: Quantum Computing\', \'description\': \'The goal of this course is to provide students in 

In [12]:
from langchain.document_loaders import DirectoryLoader
DRIVE_FOLDER = "../data/oms-central/review"
loader = DirectoryLoader(DRIVE_FOLDER, glob='**/*.json', show_progress=True, loader_cls=JSONLoader, loader_kwargs = {'jq_schema':'.props.pageProps.course.reviews[].body', 'text_content': False})
reviews = loader.load()

100%|██████████| 88/88 [00:01<00:00, 50.36it/s]


In [13]:
reviews[0]

Document(metadata={'source': '/Users/hanqihua/Desktop/learning-assistant/data/oms-central/review/CS-8803-O15.json', 'seq_num': 1}, page_content='I love the teaching team. So fun, great course that can be useful when you thinking about developing your next big ideas.')

In [1]:
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()
OPEN_API_KEY = os.getenv("OPEN_API_KEY")

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
REVIEWS_CHROMA_PATH = "../data/chroma_data/"
reviews_vector_db = Chroma.from_documents(
    reviews, OpenAIEmbeddings(openai_api_key = OPEN_API_KEY), persist_directory=REVIEWS_CHROMA_PATH
)

## Start running code from here

In [10]:
# reviews_retriever  = reviews_vector_db.as_retriever(k=10)
# BASE_SOURCE = "/Users/hanqihua/Desktop/learning-assistant/data/oms-central/review/"
# COURSE = "CS-8803-O15"
# END = ".json"
# source = BASE_SOURCE + COURSE + END
# question = """what are the positive sides about this course?"""
# relevant_docs = reviews_vector_db.similarity_search(question, k=3, filter={"source": source})
# relevant_docs

[Document(metadata={'seq_num': 1, 'source': '/Users/hanqihua/Desktop/learning-assistant/data/oms-central/review/CS-8803-O15.json'}, page_content='I love the teaching team. So fun, great course that can be useful when you thinking about developing your next big ideas.'),
 Document(metadata={'seq_num': 1, 'source': '/Users/hanqihua/Desktop/learning-assistant/data/oms-central/review/CS-8803-O15.json'}, page_content='I love the teaching team. So fun, great course that can be useful when you thinking about developing your next big ideas.'),
 Document(metadata={'seq_num': 9, 'source': '/Users/hanqihua/Desktop/learning-assistant/data/oms-central/review/CS-8803-O15.json'}, page_content='Very easy but interesting topics. Helpful TAs. Opportunities to earn extra credit. Recommend if you want an easy A.')]

In [3]:
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()
OPEN_API_KEY = os.getenv("OPEN_API_KEY")

In [7]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
REVIEWS_CHROMA_PATH = "../data/chroma_data/"
reviews_vector_db = Chroma(
    persist_directory=REVIEWS_CHROMA_PATH,
    embedding_function=OpenAIEmbeddings(openai_api_key = OPEN_API_KEY)
)
question = """I want some challenging courses?"""
relevant_docs = reviews_vector_db.similarity_search(question, k=10)
relevant_docs

[Document(metadata={'seq_num': 23, 'source': '/Users/hanqihua/Desktop/learning-assistant/data/oms-central/review/CS-6265.json'}, page_content='The course was fun and exciting'),
 Document(metadata={'seq_num': 55, 'source': '/Users/hanqihua/Desktop/learning-assistant/data/oms-central/review/CS-6262.json'}, page_content='Great course'),
 Document(metadata={'seq_num': 282, 'source': '/Users/hanqihua/Desktop/learning-assistant/data/oms-central/review/CS-7646.json'}, page_content='A good course for summer.'),
 Document(metadata={'seq_num': 213, 'source': '/Users/hanqihua/Desktop/learning-assistant/data/oms-central/review/CS-7638.json'}, page_content='Great course. The third and final project is by far the most challenging part of the course!'),
 Document(metadata={'seq_num': 149, 'source': '/Users/hanqihua/Desktop/learning-assistant/data/oms-central/review/CS-6200.json'}, page_content='Good course.'),
 Document(metadata={'seq_num': 150, 'source': '/Users/hanqihua/Desktop/learning-assistant/

In [8]:
import dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
import os

dotenv.load_dotenv()
OPEN_API_KEY = os.getenv("OPEN_API_KEY")

review_system_template_str = """Your job is to use student
reviews to answer questions about their experience at univeristy. 
Use the following context to answer questions.
Be as detailed as possible, but don't make up any information
that's not from the context. If you don't know an answer, say
you don't know.

{context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_system_template_str
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"], template="{question}"
    )
)

messages = [review_system_prompt, review_human_prompt]
review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=messages,
)
chat_model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, api_key = OPEN_API_KEY)


In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
REVIEWS_CHROMA_PATH = "../data/chroma_data/"
reviews_vector_db = Chroma(
    persist_directory=REVIEWS_CHROMA_PATH,
    embedding_function=OpenAIEmbeddings(openai_api_key = OPEN_API_KEY)
)
reviews_retriever  = reviews_vector_db.as_retriever(k=20)

review_chain = (
    {"context": reviews_retriever, "question": RunnablePassthrough()}
    | review_prompt_template
    | chat_model
    | StrOutputParser()
)

/var/folders/_k/bxqnpwnx3rbc52gzvkrlxl5m0000gn/T/ipykernel_49067/3297417579.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  reviews_vector_db = Chroma(


In [ ]:
question = """I want some interesting courses?"""
review_chain.invoke(question)

In [17]:
reviews_retriever.vectorstore

## Finished Version

In [3]:
import dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
import os

dotenv.load_dotenv()
OPEN_API_KEY = os.getenv("OPEN_API_KEY")

review_system_template_str = """Your job is to use student
reviews to answer questions about their experience at univeristy. 
Use the following context to answer questions.
Be as detailed as possible, but don't make up any information
that's not from the context. If you don't know an answer, say
you don't know.

{context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_system_template_str
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"], template="{question}"
    )
)

messages = [review_system_prompt, review_human_prompt]
review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=messages,
)
chat_model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, api_key = OPEN_API_KEY)


from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableMap
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
REVIEWS_CHROMA_PATH = "../data/chroma_data/"

# Initialize the vector database and retriever
reviews_vector_db = Chroma(
    persist_directory=REVIEWS_CHROMA_PATH,
    embedding_function=OpenAIEmbeddings(openai_api_key=OPEN_API_KEY)
)
reviews_retriever = reviews_vector_db.as_retriever(k=10)

# Define a function to process each document's content and metadata
def format_with_metadata(documents):
    return [
        f"Review: {doc.page_content}\nCourse: {doc.metadata['source'].split('/')[-1].rstrip('.json')}"
        for doc in documents["context"]
    ]

# Create a chain that incorporates the formatted documents with metadata
review_chain = (
    {"context": reviews_retriever, "question": RunnablePassthrough()}
    | RunnableMap({"context": format_with_metadata, "question": lambda x: x})  # Format context with metadata
    | review_prompt_template
    | chat_model
    | StrOutputParser()
)

question = """Give me some challenging courses"""
review_chain.invoke(question)